# Rectangular integration using MIDAS_embed

In [5]:
source("/root/workspace/code/midas/preprocess/utils.R")
setwd("/root/workspace/code/midas/")
library(RColorBrewer)

parser <- ArgumentParser()
parser$add_argument("--task", type = "character", default = "dogma_single_adt")
parser$add_argument("--method", type = "character", default = "midas_embed")
parser$add_argument("--experiment", type = "character", default = "e0")
parser$add_argument("--model", type = "character", default = "default")
parser$add_argument("--init_model", type = "character", default = "sp_00001899")
o <- parser$parse_known_args()[[1]]

config <- parseTOML("configs/data.toml")[[gsub("_vd.*|_vt.*|_transfer$|_ref_.*$", "", o$task)]]
subset_names <- basename(config$raw_data_dirs)
subset_ids <- sapply(seq_along(subset_names) - 1, toString)
input_dirs <- pj("result", o$task, o$experiment, o$model, "predict", o$init_model, paste0("subset_", subset_ids))
pp_dir <- pj("data", "processed", o$task)
output_dir <- pj("result", "comparison", o$task, o$method, o$experiment, o$model, o$init_model)
mkdir(output_dir, remove_old = F)
if (grepl("_vt", o$task)) {
    fn <- paste0("l1_", tail(strsplit(o$task, split = "_")[[1]], 1), ".csv")
    label_paths <- pj(config$raw_data_dirs, "label_seurat", fn)
} else {
    label_paths <- pj(config$raw_data_dirs, "label_seurat", "l1.csv")
}

K <- parseTOML("configs/model.toml")[["default"]]$dim_c
l <- 7.5  # figure size
L <- 10   # figure size
m <- 0.6  # legend margin

## Load preprossed data

In [6]:
z_list <- list()
cell_name_list <- list()
label_list <- list()
is_label <- T
subset_name_list <- list()
S <- length(subset_names)
for (i in seq_along(subset_names)) {
    subset_name <- subset_names[i]
    z_dir    <- pj(input_dirs[i], "z", "joint")
    fnames <- dir(path = z_dir, pattern = ".csv$")
    fnames <- str_sort(fnames, decreasing = F)

    z_subset_list <- list()
    N <- length(fnames)
    for (n in seq_along(fnames)) {
        message(paste0("Loading Subset ", i, "/", S, ", File ", n, "/", N))
        z_subset_list[[n]] <- read.csv(file.path(z_dir, fnames[n]), header = F)
    }
    z_list[[subset_name]] <- bind_rows(z_subset_list)

    cell_name_list[[subset_name]] <- read.csv(pj(pp_dir, paste0("subset_", subset_ids[i]),
        "cell_names.csv"), header = T)[, 2]
    if (file.exists(label_paths[i])) {
        label_list[[subset_name]] <- read.csv(label_paths[i], header = T)[, 2]
    } else {
        is_label <- F
    }
    
    subset_name_list[[subset_name]] <- rep(subset_name, length(cell_name_list[[subset_name]]))
}

Loading Subset 1/4, File 1/29

Loading Subset 1/4, File 2/29

Loading Subset 1/4, File 3/29

Loading Subset 1/4, File 4/29

Loading Subset 1/4, File 5/29

Loading Subset 1/4, File 6/29

Loading Subset 1/4, File 7/29

Loading Subset 1/4, File 8/29

Loading Subset 1/4, File 9/29

Loading Subset 1/4, File 10/29

Loading Subset 1/4, File 11/29

Loading Subset 1/4, File 12/29

Loading Subset 1/4, File 13/29

Loading Subset 1/4, File 14/29

Loading Subset 1/4, File 15/29

Loading Subset 1/4, File 16/29

Loading Subset 1/4, File 17/29

Loading Subset 1/4, File 18/29

Loading Subset 1/4, File 19/29

Loading Subset 1/4, File 20/29

Loading Subset 1/4, File 21/29

Loading Subset 1/4, File 22/29

Loading Subset 1/4, File 23/29

Loading Subset 1/4, File 24/29

Loading Subset 1/4, File 25/29

Loading Subset 1/4, File 26/29

Loading Subset 1/4, File 27/29

Loading Subset 1/4, File 28/29

Loading Subset 1/4, File 29/29

Loading Subset 2/4, File 1/24

Loading Subset 2/4, File 2/24

Loading Subset 2/4,

## Create seurat object

In [7]:
rna <- t(data.matrix(bind_rows(z_list))) * 0  # pseudo rna counts
colnames(rna) <- do.call("c", unname(cell_name_list))
rownames(rna) <- paste0("rna-", seq_len(nrow(rna)))
obj <- CreateSeuratObject(counts = rna, assay = "rna")

z <- data.matrix(bind_rows(z_list))
c <- z[, 1:K]
colnames(c) <- paste0("c_", seq_len(ncol(c)))
rownames(c) <- colnames(obj)
obj[["c"]] <- CreateDimReducObject(embeddings = c, key = "c_", assay = "rna")

u <- z[, (K+1):(K+2)]
colnames(u) <- paste0("u_", seq_len(ncol(u)))
rownames(u) <- colnames(obj)
obj[["u"]] <- CreateDimReducObject(embeddings = u, key = "u_", assay = "rna")

obj@meta.data$batch <- factor(x = do.call("c", unname(subset_name_list)), levels = subset_names)
table(obj@meta.data$batch)[unique(obj@meta.data$batch)]
if (is_label) {
    obj@meta.data$l1 <- do.call("c", unname(label_list))
} else {
    obj@meta.data$l1 <- obj@meta.data$batch
}

obj


lll_ctrl lll_stim dig_ctrl dig_stim 
    7361     5897    10190     9527 

An object of class Seurat 
34 features across 32975 samples within 1 assay 
Active assay: rna (34 features, 0 variable features)
 2 dimensional reductions calculated: c, u

## Visualization

In [8]:
obj <- RunUMAP(obj, reduction = 'c', dims = 1:K, reduction.name = 'c.umap')
obj <- RunUMAP(obj, reduction = 'u', dims = 1:2, metric = "euclidean", reduction.name = 'u.umap')
SaveH5Seurat(obj, pj(output_dir, "obj.h5seurat"), overwrite = TRUE)


# obj <- LoadH5Seurat(pj(output_dir, "obj.h5seurat"), reductions = c("c.umap", "u.umap"))

dim_plot(obj, w = S*l, h = l+0.8, reduction = 'c.umap', no_axes = T, border = T, raster = T, raster_dpi = 500,
    split.by = "batch", group.by = "batch", label = F,
    repel = T, label.size = 4, pt.size = 1, cols = col_4,
    title = paste0(o$method, ", ", o$task), legend = F,
    save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "c_split_batch", sep = "_")))

dim_plot(obj, w = S*l+m, h = l+0.8, reduction = 'c.umap', no_axes = T, border = T, raster = T, raster_dpi = 500,
    split.by = "batch", group.by = "l1", label = F,
    repel = T, label.size = 4, pt.size = 1, cols = col_8,
    title = paste0(o$method, ", ", o$task), legend = T,
    save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "c_split_label", sep = "_")))

# dim_plot(obj, w = L+m, h = L, reduction = 'c.umap',
#     split.by = NULL, group.by = "batch", label = F,
#     repel = T, label.size = 4, pt.size = 0.1, cols = bcols,
#     title = o$method, legend = T,
#     save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "c_merged_batch", sep = "_")))

# dim_plot(obj, w = L+m, h = L, reduction = 'c.umap',
#     split.by = NULL, group.by = "l1", label = F,
#     repel = T, label.size = 4, pt.size = 0.1, cols = dcols,
#     title = o$method, legend = T,
#     save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "c_merged_label", sep = "_")))

# dim_plot(obj, w = L+m, h = L, reduction = 'u.umap',
#     split.by = NULL, group.by = "batch", label = F,
#     repel = T, label.size = 4, pt.size = 0.1, cols = bcols,
#     title = o$method, legend = T,
#     save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "u_merged_batch", sep = "_")))

# dim_plot(obj, w = L+m, h = L, reduction = 'u.umap',
#     split.by = NULL, group.by = "l1", label = F,
#     repel = T, label.size = 4, pt.size = 0.1, cols = dcols,
#     title = o$method, legend = T,
#     save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "u_merged_label", sep = "_")))


# obj <- LoadH5Seurat(pj(output_dir, "obj.h5seurat"), reductions = c("c.umap", "u.umap"))

dim_plot(obj, title = rename_task(o$task), w = L, h = L+m, reduction = 'c.umap', no_axes = T, border = T, raster = T, raster_dpi = 500,
    split.by = NULL, group.by = "batch", label = F, repel = T, label.size = 4, pt.size = 0.1, cols = col_4, legend = F,
    save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "c_merged_batch", sep = "_")))

dim_plot(obj, title = rename_task(o$task), w = L, h = L+m, reduction = 'c.umap', no_axes = T, border = T, raster = T, raster_dpi = 500,
    split.by = NULL, group.by = "l1", label = F, repel = T, label.size = 4, pt.size = 0.1, cols = col_8, legend = F,
    save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "c_merged_label", sep = "_")))

dim_plot(obj, title = rename_task(o$task), w = L, h = L+m, reduction = 'u.umap', no_axes = T, border = T, raster = T, raster_dpi = 500,
    split.by = NULL, group.by = "batch", label = F,  repel = T, label.size = 4, pt.size = 0.1, cols = col_4, legend = F,
    save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "u_merged_batch", sep = "_")))

dim_plot(obj, title = rename_task(o$task), w = L, h = L+m, reduction = 'u.umap', no_axes = T, border = T, raster = T, raster_dpi = 500,
    split.by = NULL, group.by = "l1", label = F, repel = T, label.size = 4, pt.size = 0.1, cols = col_8, legend = F,
    save_path = pj(output_dir, paste(o$task, o$method, o$experiment, o$model, o$init_model, "u_merged_label", sep = "_")))

08:12:35 UMAP embedding parameters a = 0.9922 b = 1.112

08:12:35 Read 32975 rows and found 32 numeric columns

08:12:35 Using Annoy for neighbor search, n_neighbors = 30

08:12:35 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

08:12:39 Writing NN index file to temp file /tmp/RtmpcB09ZU/file53fc43264b98

08:12:39 Searching Annoy index using 64 threads, search_k = 3000

08:12:39 Annoy recall = 100%

08:12:41 Commencing smooth kNN distance calibration using 64 threads
 with target n_neighbors = 30

08:12:43 Initializing from normalized Laplacian + noise (using irlba)

08:12:43 Commencing optimization for 200 epochs, with 1355250 positive edges

